# Hamiltonian Monte Carlo

In [ ]:
Hamiltonian Monte Carlo (HMC) is an extension of Metropolis-Hastings(MH).



The algorithm breaks down into four parts:

1. Set up: Take the previous position and copy, such that you have q0 and q1. Randomly sample a momentum from N(0,1) and copy, such that you have p0 and p1. Find the gradient of the PDF with respect to position -(x-mu)/sigma^2 for a single variable gaussian.
2. Leapfrog: Use leapfrog integration to update q1 and p1 (ie Hamiltonian motion or a particle.) In practice, this is the most sensitive part; small adjustments can induce unstable behavior.
3. MH Dance: Lastly, multiply p1 by (-1), this ensures reversibility (ie q1 can be reached from q0 given momentum p0 AND q0 can be reached from q1 given momentum -p1). This gives us the information we need for the metropolis-hastings “dance”. Keep in mind we’re using negative log probabilities, so the math is all in terms of addition and subtraction. Then accept/reject movement.
4. Repeat for fixed number of iterations.


In [ ]:
import numpy as np
import random
import scipy.stats as st
import matplotlib.pyplot as plt

def normal(x,mu,sigma):
    numerator = np.exp(-1*((x-mu)**2)/(2*sigma**2))
    denominator = sigma * np.sqrt(2*np.pi)
    return numerator/denominator

def neg_log_prob(x,mu,sigma):
    return -1*np.log(normal(x=x,mu=mu,sigma=sigma))

def HMC(mu=0.0,sigma=1.0,path_len=1,step_size=0.25,initial_position=0.0,epochs=1_000):
    # setup
    steps = int(path_len/step_size) # path_len and step_size are tricky parameters to tune...
    samples = [initial_position]
    momentum_dist = st.norm(0, 1) 
    # generate samples
    for e in range(epochs):
        q0 = np.copy(samples[-1])
        q1 = np.copy(q0)
        p0 = momentum_dist.rvs()        
        p1 = np.copy(p0) 
        dVdQ = -1*(q0-mu)/(sigma**2) # gradient of PDF wrt position (q0) aka potential energy wrt position

        # leapfrog integration begin
        for s in range(steps): 
            p1 += step_size*dVdQ/2 # as potential energy increases, kinetic energy decreases, half-step
            q1 += step_size*p1 # position increases as function of momentum 
            p1 += step_size*dVdQ/2 # second half-step "leapfrog" update to momentum    
        # leapfrog integration end        
        p1 = -1*p1 #flip momentum for reversibility     

        
        #metropolis acceptance
        q0_nlp = neg_log_prob(x=q0,mu=mu,sigma=sigma)
        q1_nlp = neg_log_prob(x=q1,mu=mu,sigma=sigma)        

        p0_nlp = neg_log_prob(x=p0,mu=0,sigma=1)
        p1_nlp = neg_log_prob(x=p1,mu=0,sigma=1)
        
        # Account for negatives AND log(probabiltiies)...
        target = q0_nlp - q1_nlp # P(q1)/P(q0)
        adjustment = p1_nlp - p0_nlp # P(p1)/P(p0)
        acceptance = target + adjustment 
        
        event = np.log(random.uniform(0,1))
        if event <= acceptance:
            samples.append(q1)
        else:
            samples.append(q0)
    
    return samples 

In [ ]:
import matplotlib.pyplot as plt
mu = 0
sigma = 1
trial = HMC(mu=mu,sigma=sigma,path_len=1.5,step_size=0.25)

lines = np.linspace(-6,6,10_000)
normal_curve = [normal(x=l,mu=mu,sigma=sigma) for l in lines]

plt.plot(lines,normal_curve)
plt.hist(trial,density=True,bins=20)
plt.show() 

## Resources

* https://towardsdatascience.com/python-hamiltonian-monte-carlo-from-scratch-955dba96a42d
* Simulation-based Probabilistic Risk Assessment : https://arxiv.org/pdf/2207.12575.pdf
* https://zhijingeu.medium.com/building-a-probabilistic-risk-estimate-using-monte-carlo-simulations-with-python-mcerp-7d57e63112fa
* https://towardsdatascience.com/python-hamiltonian-monte-carlo-from-scratch-955dba96a42d